#### Loading Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('irrigation_machine.csv')

In [3]:
data.head()

,Unnamed: 0,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,parcel_0,parcel_1,parcel_2
0,0,1.0,2.0,1.0,7.0,0.0,1.0,1.0,4.0,0.0,...,8.0,1.0,0.0,2.0,1.0,9.0,2.0,0,1,0
1,1,5.0,1.0,3.0,5.0,2.0,2.0,1.0,2.0,3.0,...,4.0,5.0,5.0,2.0,2.0,2.0,7.0,0,0,0
2,2,3.0,1.0,4.0,3.0,4.0,0.0,1.0,6.0,0.0,...,3.0,3.0,1.0,0.0,3.0,1.0,0.0,1,1,0
3,3,2.0,2.0,4.0,3.0,5.0,0.0,3.0,2.0,2.0,...,4.0,1.0,1.0,4.0,1.0,3.0,2.0,0,0,0
4,4,4.0,3.0,3.0,2.0,5.0,1.0,3.0,1.0,1.0,...,1.0,3.0,2.0,2.0,1.0,1.0,0.0,1,1,0


In [4]:
data.dtypes

Unnamed: 0      int64
sensor_0      float64
sensor_1      float64
sensor_2      float64
sensor_3      float64
sensor_4      float64
sensor_5      float64
sensor_6      float64
sensor_7      float64
sensor_8      float64
sensor_9      float64
sensor_10     float64
sensor_11     float64
sensor_12     float64
sensor_13     float64
sensor_14     float64
sensor_15     float64
sensor_16     float64
sensor_17     float64
sensor_18     float64
sensor_19     float64
parcel_0        int64
parcel_1        int64
parcel_2        int64
dtype: object

In [5]:
# Describe the data
print('Dataset stats: \n', data.describe())

Dataset stats: 
         Unnamed: 0     sensor_0     sensor_1     sensor_2     sensor_3  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000   
mean    999.500000     1.437000     1.659000     2.654500     2.674500   
std     577.494589     1.321327     1.338512     1.699286     1.855875   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%     499.750000     0.000000     1.000000     1.000000     1.000000   
50%     999.500000     1.000000     1.000000     2.000000     2.000000   
75%    1499.250000     2.000000     2.000000     4.000000     4.000000   
max    1999.000000     8.000000     9.000000    10.000000    11.000000   

          sensor_4     sensor_5     sensor_6     sensor_7     sensor_8  ...  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000  ...   
mean      2.887500     1.411000     3.315500     4.201500     1.214000  ...   
std       1.816451     1.339394     2.206444     2.280241     1.386782  ...   


In [6]:
# Count the number of observations of each class
print('Observations per class 0: \n', data['parcel_0'].value_counts())
print('Observations per class 1: \n', data['parcel_1'].value_counts())
print('Observations per class 2: \n', data['parcel_2'].value_counts())

Observations per class 0: 
 1    1271
0     729
Name: parcel_0, dtype: int64
Observations per class 1: 
 1    1461
0     539
Name: parcel_1, dtype: int64
Observations per class 2: 
 0    1576
1     424
Name: parcel_2, dtype: int64


#### Data preparation

In [7]:
from sklearn.model_selection import train_test_split
y = data[['parcel_0','parcel_1','parcel_2']]
X = data.drop(['parcel_0','parcel_1','parcel_2','Unnamed: 0'], axis=1)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

#### Model configuration

In [8]:
# Import the Sequential model and Dense layer
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Creates a model given an activation and learning rate
def create_model(learning_rate=0.01, activation='relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)

    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(128, input_shape=(30,), activation=activation))
    model.add(Dense(256, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    # Compile your model with your optimizer, loss, and metrics
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

Using TensorFlow backend.


#### Model compilation

In [9]:
# Import KerasClassifier from keras wrappers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Create a KerasClassifier
model = KerasClassifier(build_fn = create_model)

# Define the parameters to try out
params = {'activation': ['relu', 'tanh'], 
          'batch_size': [32, 128, 256], 
          'epochs': [50, 100, 200], 
          'learning_rate': [0.1, 0.01, 0.001]}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model, param_distributions = params, cv = 3)

In [10]:
random_search

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000250455AF5C0>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'activation': ['relu', 'tanh'],
                                        'batch_size': [32, 128, 256],
                                        'epochs': [50, 100, 200],
                                        'learning_rate': [0.1, 0.01, 0.001]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [11]:
# Import KerasClassifier from keras wrappers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Create a KerasClassifier
model = KerasClassifier(build_fn = create_model, epochs = 50, batch_size = 128, verbose = 0)

# Calculate the accuracy score for each fold
kfolds = cross_val_score(model, X, y, cv = 3)

# Print the mean accuracy
print('The mean accuracy was:', kfolds.mean())

# Print the accuracy standard deviation
print('With a standard deviation of:', kfolds.std())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Error when checking input: expected dense_1_input to have shape (30,) but got array with shape (20,)

  FitFailedWarning)
C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Error when checking input: expected dense_4_input to have shape (30,) but got array with shape (20,)

  FitFailedWarning)


The mean accuracy was: nan
With a standard deviation of: nan


C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Error when checking input: expected dense_7_input to have shape (30,) but got array with shape (20,)

  FitFailedWarning)
